# Running the NUMAR model



Original paper: Chen, Ronghua & Twilley, Robert. (1999). A Simulation Model of Organic Matter and Nutrient Accumulation in Mangrove Wetland Soils. Biogeochemistry. 44. 93-118. 10.1007/BF00993000. https://www.researchgate.net/publication/226714484_A_Simulation_Model_of_Organic_Matter_and_Nutrient_Accumulation_in_Mangrove_Wetland_Soils
- Python Conversion: Tom Kaiser
- Python Wrapper and Notebook: Alex Christensen
- Model Updates For Marsh: Andy Cassaway and Pradipta Biswas




# 1a. Import Python Packages

Uncomment the following cell if running in Google Colab

In [1]:
## if running in Google Colab, uncomment the lines below

!pip install rasterio
!pip install earthaccess
!pip install cartopy

!apt install gdal-bin
# !gdalinfo --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
    Uninstalling fsspec-2023.6.0:
      Successfully uninstalled fsspec-2023.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_

In [2]:
import sys
import os
from pathlib import Path
import rasterio
import matplotlib.pyplot as plt
from  matplotlib.colors import ListedColormap, BoundaryNorm
from  matplotlib.cm import ScalarMappable
import math
import pandas as pd
import earthaccess
import cartopy.crs as ccrs
import imageio
from osgeo import gdal, ogr
import numpy as np

earthaccess.login()



Enter your Earthdata Login username: achri
Enter your Earthdata password: ··········


# 1b. Set working directories

Uncomment the following cell if you are running in Google Colab

In [3]:
if os.path.isdir('/content/deltax_workshop_2024'):
  !rm -r /content/deltax_workshop_2024
! git clone https://github.com/achri19/deltax_workshop_2024.git

%cd /content/deltax_workshop_2024/tutorials/6_NUMARModel/Landscape_example
sys.path.append('/content/deltax_workshop_2024/tutorials/6_NUMARModel/src')


Cloning into 'deltax_workshop_2024'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 222 (delta 71), reused 154 (delta 40), pack-reused 0
Receiving objects: 100% (222/222), 30.21 MiB | 25.40 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/deltax_workshop_2024/tutorials/6_NUMARModel/Landscape_example


In [4]:
example_dir = Path(os.getcwd())
repo_dir = example_dir.parent.absolute()
script_dir = repo_dir / 'src'
working_dir = repo_dir / 'data'
tmp_dir = example_dir / 'TMP'
Path(tmp_dir).mkdir(parents=True, exist_ok=True)
sys.path.insert(1,str(script_dir))


# 1c. Import NUMAR model

In [5]:
from landscape import run_numar
from input_files import *

In [6]:
Path(tmp_dir).mkdir(parents=True, exist_ok=True)
ref_file = example_dir / 'aoi_4326.shp'
res = 100
AOI = gpd.read_file(ref_file)
bounds_4326 =  AOI.total_bounds

x1,y1,x2,y2 = math.floor(bounds_4326[0]),math.floor(bounds_4326[3]),math.floor(bounds_4326[2]),math.floor(bounds_4326[1])

zone = int(np.ceil((bounds_4326[0] + 180)/6))

if y1>=0 and y2>=0:
    NS = 'n'
    EPSG = 32200+zone
elif y1>=0 and y2<0:
    NS = 'n'
    NS2 = 's'
    EPSG = 32200+zone
else:
    NS = 's'
    y = abs(y1)
    EPSG = 32700+zone
if x1>=0:
    EW = 'e'
elif x1<0 and x2>=0:
    EW = 'e'
    EW2 = 'w'
else:
    EW = 'w'
    x = abs(x1)
print('UTM Zone: %s%s EPSG: %s' %(zone,NS,EPSG))

AOI_utm = AOI.to_crs(EPSG)
AOI_utm.to_file(tmp_dir / ('aoi_%s.shp' %(EPSG)))
os.system("gdal_rasterize -burn 0 -tr %s %s %s %s" %(res,res,tmp_dir/('aoi_%s.shp'%(EPSG)),tmp_dir / ('aoi_%s.tif'%(EPSG))))



UTM Zone: 15n EPSG: 32215


0

In [7]:
ref_file = tmp_dir / ('aoi_%s.tif' %(EPSG))
AOI = rasterio.open(ref_file)

profile = AOI.profile


## Get Input Rasters

### You need to download the following files:
- ### Delta-X AVIRIS-NG Aboveground Biomass (spring and Fall)
    - ### https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=2138
- ### ESA WorldCover 2021 Landcover Maps
    - ### https://worldcover2021.esa.int/downloader
- ### CPRA Vegetation
    - ### https://cims.coastal.la.gov/Viewer/GISDownload.aspx
    - ### direct download = https://cims.coastal.louisiana.gov/Viewer/metadata/zips/vegtype2021.zip
- ### Delta-X Delft3D Annual inorganic mass accumulation rate (IMAR): annual rates of mineral mass deposited on wetlands, estimated from analysis of storm frequency and deposition.:
    - ### https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=2302

In [8]:
AGB, SALINITY, LANDCOVER, IMAR, BASINS, CLASSES, mask, lats,lons = get_input_files(AOI,example_dir,working_dir,tmp_dir,EPSG,bounds_4326)



##### ABOVEGROUND BIOMASS



##### ESA WorldCover landcover type



##### CPRA Vegetation Type (with Salinity Zones)


##### Modified Hydrologic Basins


##### Delft3D Inorganice Mass Accumulation Rate (IMAR)


##### Delta-X Watermask


## Get Input Parameters

### From Delft3D Model IMAR output:
- si = inorganic matter deposition rate on the soil surface (g/cm2/yr)
- oms = organic matter deposition rate on the soil surfcae (g/cm2/yr)

### From AVIRIS-NG Biomass data products:
- r0 = root biomass at the surface (g/cm2)

### Based on landcover type:
- b0 = self-packing density of organic matter (g/cm3)
- bi = self-packing density of inorganic matter (g/cm3)
- c0 = lignin content in the surface deposit (g/g)
- c1 = ash content in the root biomass (g/g)
- c2 = cellulose content in the surface deposit (g/g)
- c4 = cellulose content in the root biomass(g/g)
- fc1 = lignin content in the root biomass
- kb = belowground decomposition rate of labile organic matter (1/yr)
- kc = cellulos decomposition rate (1/yr)
- kl = lignin decomposition rate (1/yr)
- kr = root turnover rate (1/yr)
- e = root attenuation rate

In [ ]:
fig, [[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2,figsize = (20,10))

im1 = ax1.imshow(AGB*mask,vmin=0,vmax=15,interpolation='nearest')
ax1.set_title('AVIRIS-NG Biomass (Mg/Ha)')
fig.colorbar(im1,ax = ax1)


colors = ['#00a000','#966400','#ffb400','#ffff64','#c31400','#fff5d7','#ffffff','#0046c8','#00dc82','#009678','#ffebaf']
values = [10,20,30,40,50,60,70,80,90,95,100]
labels = ["Tree cover","Shrubland","Grassland","Cropland","Built-up","Bare / sparse vegetation","Snow and ice","Permanent water bodies","Herbaceous wetland","Mangroves","Moss and lichen"]
bounds = np.append(values, values[-1] + 2)
ticks = [15,25,35,45,55,65,75,85,93,97,101]
cmap = ListedColormap(colors)
norm = BoundaryNorm(bounds, ncolors=len(colors))
im2 = ax2.imshow(LANDCOVER*mask,cmap=cmap,norm=norm,interpolation='nearest')
ax2.set_title('WorldCover 2021')
cbar = fig.colorbar(im2,ax=ax2,ticks = ticks)
cbar.ax.set_yticklabels(labels)  # horizontal colorbar


colors = ['#ffff00','#894444','#4ce600','#ff00c5','#73ffdf','#ff0000','#0070ff']
values =[1,2,3,4,5,6,7]
labels = ['Other/Urban','Forested Wetland','Fresh Marsh','Intermed Marsh','Brackish Marsh','Saline Marsh','Open Water']
bounds = np.append(values, values[-1] + 1)
ticks = [1.5,2.5,3.5,4.5,5.5,6.5,7.5]
cmap = ListedColormap(colors)
norm = BoundaryNorm(bounds, ncolors=len(colors))
im3 = ax3.imshow(SALINITY*mask,cmap=cmap,norm=norm,interpolation='nearest')
cbar = fig.colorbar(im3,ax=ax3,ticks = ticks)
cbar.ax.set_yticklabels(labels)  # horizontal colorbar
ax3.set_title('Salinity Type')


im4 = ax4.imshow(IMAR*mask,vmin=0,vmax=.5,interpolation='nearest')
ax4.set_title('Mineral Deposition (g/cm2/yr)')
fig.colorbar(im4,ax = ax4)


## Run NUMAR model for 10 years

In [ ]:
run_config = pd.DataFrame({
                    'lat':lats.flatten(),
                    'long':lons.flatten()
                        })

In [ ]:
parameters_df = pd.read_csv(script_dir/'parameters_by_class.csv')
parameters = parameters_df.columns


In [ ]:
for parameter in parameters[2:14]:
    print(parameter)
    lut = parameters_df[parameter].values
    classes2 = np.where(np.isnan(CLASSES),0,CLASSES).astype(int)
    output = lut[classes2]*mask
    run_config[parameter] = output.flatten()

In [ ]:
classes2

In [ ]:
oms = calculate_organicmatter_loading(IMAR,classes2,mask,parameters_df['omssi'].values)
run_config['oms'] = oms.flatten()
run_config['si'] = MORPHO.flatten()

r0 = calculate_belowground_biomass(AGB,mask,classes2,parameters_df['e'].values,parameters_df['bgbagb'].values)
run_config['r0'] = r0.flatten()

In [ ]:
run_config = run_config.dropna().reset_index()
run_config['id'] = run_config.index

print('There are %s pixels' %(len(run_config)))

In [ ]:
run_config[:].to_csv(example_dir/'NUMAR_Landscape_Input.csv')

The next cell will run the model. We are running at a reduced resolution (100m) to allow faster computation time. In Google Colab, this will take 3-4 hours.

In [ ]:
year100 = run_numar(example_dir/'NUMAR_Landscape_Input_%s.csv' %(res),example_dir/'NUMAR_Landscape_Results_%s.csv' %(res))

In [ ]:
# year100 = pd.read_csv(example_dir/'NUMAR_Landscape_Results_%s.csv' %(res))
year100

In [ ]:
for year in np.arange(10,110,10):
    fig, ax= plt.subplots(1,1, subplot_kw={'projection': ccrs.epsg(EPSG)})
    sc = ax.scatter(year100['long'],year100['lat'],s=0.005,vmin=0,vmax=3, c=year100['Total Accretion %syrs (cm)' %(year)]/year)
    plt.colorbar(sc,label='Mean Accretion Rate (cm/yr)',shrink=0.3)
    plt.title('Year: ' + str(year))
    ax.set_extent([AOI.bounds.left,AOI.bounds.right,AOI.bounds.bottom,AOI.bounds.top], crs=ccrs.epsg(EPSG))
    fig.tight_layout()
    fig.savefig(example_dir/('MeanAccretion_Year%s.png'%(str(year).zfill(3))))
    plt.close()
fig, ax= plt.subplots(1,1, subplot_kw={'projection': ccrs.epsg(EPSG)})
sc = ax.scatter(year100['long'],year100['lat'],s=0.005,vmin=0,vmax=3, c=year100['Mean Accretion (cm/yr)'])
plt.colorbar(sc,label='Mean Accretion Rate (cm/yr)',shrink=0.3)
plt.title('Year: ' + str(year))
ax.set_extent([AOI.bounds.left,AOI.bounds.right,AOI.bounds.bottom,AOI.bounds.top], crs=ccrs.epsg(EPSG))
fig.tight_layout()
fig.savefig(example_dir/('MeanAccretion_100yr.png'))

In [ ]:
all_figs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(example_dir)
                for f in fnmatch.filter(files,'MeanAccretion_Year*.png' )]
all_figs.sort()
with imageio.get_writer(example_dir / ('MeanAccretion_all.gif' ),mode='I',duration=500) as writer:
    for file in all_figs:
        image = imageio.imread(file)
        writer.append_data(image)



In [ ]:
os.system('rm -r %s' %(tmp_dir))